<a href="https://colab.research.google.com/github/Nithin46/Project_Dummy/blob/main/IMDB_Ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import plotly.express as px


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
Net = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/netflix_titles.csv', low_memory=False)
Net.head(3)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,23-Dec-16,2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,20-Dec-18,2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."


In [46]:
IMDB = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/IMDb movies.csv',low_memory=False)
IMDB.head(3)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0


In [47]:
new_ratings = pd.DataFrame({'Title':IMDB.title,
                    'Rating': IMDB.avg_vote,
                           'Year' : IMDB.year})

new_ratings.drop_duplicates(subset=['Title','Year','Rating'], inplace=True)

print(new_ratings.shape)
new_ratings.head(5)

(85852, 3)


,Title,Rating,Year
0,Miss Jerry,5.9,1894
1,The Story of the Kelly Gang,6.1,1906
2,Den sorte drøm,5.8,1911
3,Cleopatra,5.2,1912
4,L'Inferno,7.0,1911


In [48]:
Inner_join_data = new_ratings.merge(Net,left_on='Title', right_on='title', how='inner')
Inner_join_data=Inner_join_data.sort_values(by='Rating', ascending=False)

print(Inner_join_data.shape)
Inner_join_data.head(5)

(2739, 15)


,Title,Rating,Year,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
991,Innocent,9.0,2017,s3009,TV Show,Innocent,Seren Yüce,"Ali Atay, Haluk Bilginer, Nur Sürer, Okan Yala...",Turkey,23-Jan-19,2017,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Dramas","In a peaceful, rustic town, a retired officer ..."
1002,Breakout,9.0,2019,s1093,TV Show,Breakout,NaN,"Jeanette Aw, Elvin Ng, Zhou Ying, Christopher ...",NaN,1-Jul-17,2010,TV-14,1 Season,"International TV Shows, TV Dramas",After losing her family in a suspicious car cr...
398,Pulp Fiction,8.9,1994,s5003,Movie,Pulp Fiction,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Uma Thurman,...",United States,1-Jan-19,1994,R,154 min,"Classic Movies, Cult Movies, Dramas",This stylized crime caper weaves together stor...
380,Schindler's List,8.9,1993,s5431,Movie,Schindler's List,Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",United States,1-Apr-18,1993,R,195 min,"Classic Movies, Dramas",Oskar Schindler becomes an unlikely humanitari...
1301,Inception,8.8,2010,s2980,Movie,Inception,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...","United States, United Kingdom",1-Jan-20,2010,PG-13,148 min,"Action & Adventure, Sci-Fi & Fantasy, Thrillers","In this mind-bending sci-fi thriller, a man ru..."


In [49]:
New_Data = pd.DataFrame({'Title':Inner_join_data.title,
                    'Rating': Inner_join_data.Rating,
                    'Type': Inner_join_data.type,
                           'Year' : Inner_join_data.release_year})

New_Data.drop_duplicates(subset=['Title','Rating', 'Type', 'Year'], inplace=True)
print(New_Data.shape)
New_Data.head(5)

(2721, 4)


,Title,Rating,Type,Year
991,Innocent,9.0,TV Show,2017
1002,Breakout,9.0,TV Show,2010
398,Pulp Fiction,8.9,Movie,1994
380,Schindler's List,8.9,Movie,1993
1301,Inception,8.8,Movie,2010


In [50]:
Movies_Data = New_Data[New_Data.Type == 'Movie']
TV_Data = New_Data[New_Data.Type == 'TV Show']
print(Movies_Data.shape)
TV_Data.shape

(2384, 4)


(337, 4)

In [51]:
Movies_Data = Movies_Data.drop(['Type', 'Year'], axis=1)

In [52]:
Movies_Data

,Title,Rating
398,Pulp Fiction,8.9
380,Schindler's List,8.9
1301,Inception,8.8
1669,Much Ado About Nothing,8.6
227,Gol Maal,8.6
...,...,...
876,End Game,1.9
582,Himmatwala,1.7
2193,Aerials,1.6
1205,Pink,1.6


In [53]:
Movies_Data['Polarity_Rating'] = Movies_Data['Rating'].apply(lambda x: 'Positive' if x > 6 else 'Negative')
Movies_Data

,Title,Rating,Polarity_Rating
398,Pulp Fiction,8.9,Positive
380,Schindler's List,8.9,Positive
1301,Inception,8.8,Positive
1669,Much Ado About Nothing,8.6,Positive
227,Gol Maal,8.6,Positive
...,...,...,...
876,End Game,1.9,Negative
582,Himmatwala,1.7,Negative
2193,Aerials,1.6,Negative
1205,Pink,1.6,Negative


In [54]:
fig = px.pie(values=Movies_Data['Polarity_Rating'].value_counts(), 
             names=Movies_Data['Polarity_Rating'].value_counts().index)
fig.show()

In [55]:
Positive = Movies_Data[Movies_Data['Polarity_Rating'] == 'Positive']
Negative = Movies_Data[Movies_Data['Polarity_Rating'] == 'Negative']

print(Positive.shape)
Negative.shape

(1206, 3)


(1178, 3)

In [56]:
df = Movies_Data[['Title','Polarity_Rating']]
df

,Title,Polarity_Rating
398,Pulp Fiction,Positive
380,Schindler's List,Positive
1301,Inception,Positive
1669,Much Ado About Nothing,Positive
227,Gol Maal,Positive
...,...,...
876,End Game,Negative
582,Himmatwala,Negative
2193,Aerials,Negative
1205,Pink,Negative


In [57]:
one_hot = pd.get_dummies(df["Polarity_Rating"])
df.drop(['Polarity_Rating'],axis=1,inplace=True)
df = pd.concat([df,one_hot],axis=1)
df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Title,Negative,Positive
398,Pulp Fiction,0,1
380,Schindler's List,0,1
1301,Inception,0,1
1669,Much Ado About Nothing,0,1
227,Gol Maal,0,1
...,...,...,...
876,End Game,1,0
582,Himmatwala,1,0
2193,Aerials,1,0
1205,Pink,1,0


In [58]:
X = df['Title'].values
y = df.drop('Title', axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [59]:
X_train

array(['Golmaal: Fun Unlimited', 'Red Christmas', 'The Model', ...,
       'Dark Waters', 'World Trade Center', 'Lust Stories'], dtype=object)

In [60]:
y_train

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [61]:
vect = CountVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [62]:
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train = X_train.toarray()
X_test = X_test.toarray()

In [65]:
model = Sequential()

model.add(Dense(units=12673,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=4000,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=500,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=2, activation='sigmoid'))

opt=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [66]:
model.fit(x=X_train, y=y_train, batch_size=200, epochs=100, validation_data=(X_test, y_test), verbose=1, callbacks=early_stop)

Epoch 1/100
10/10 [==============================] - 4s 129ms/step - loss: 0.7001 - accuracy: 0.5118 - val_loss: 0.6945 - val_accuracy: 0.5367
Epoch 2/100
10/10 [==============================] - 1s 92ms/step - loss: 0.6137 - accuracy: 0.6899 - val_loss: 0.8731 - val_accuracy: 0.5346
Epoch 3/100
10/10 [==============================] - 1s 90ms/step - loss: 0.2695 - accuracy: 0.9035 - val_loss: 1.4506 - val_accuracy: 0.4969
Epoch 00003: early stopping


In [67]:
model_score = model.evaluate(X_test, y_test, batch_size=64, verbose=1)
print('Test accuracy:', model_score[1])

8/8 [==============================] - 0s 14ms/step - loss: 1.4506 - accuracy: 0.4969
Test accuracy: 0.49685534834861755


In [68]:
preds = model.predict(X_test)
preds

array([[0.53357327, 0.39042872],
       [0.9016719 , 0.06655823],
       [0.16573182, 0.7900198 ],
       [0.54132855, 0.43884817],
       [0.54132855, 0.43884817],
       [0.7527511 , 0.19449466],
       [0.54132855, 0.43884817],
       [0.07484142, 0.8980285 ],
       [0.10242233, 0.862663  ],
       [0.54132855, 0.43884817],
       [0.5050116 , 0.41891927],
       [0.54132855, 0.43884817],
       [0.8395019 , 0.11876816],
       [0.07460161, 0.8980781 ],
       [0.97635776, 0.01173779],
       [0.54132855, 0.43884817],
       [0.54132855, 0.43884817],
       [0.54132855, 0.43884817],
       [0.59234715, 0.33536902],
       [0.54132855, 0.43884817],
       [0.54756194, 0.37057662],
       [0.54132855, 0.43884817],
       [0.07568342, 0.89809656],
       [0.76960456, 0.17669675],
       [0.80778795, 0.1448675 ],
       [0.5050116 , 0.41891927],
       [0.47025478, 0.45683226],
       [0.69643366, 0.2410907 ],
       [0.54132855, 0.43884817],
       [0.40695605, 0.52587074],
       [0.